In [1]:
2

2

In [2]:
import seaborn as sns

import sklearn
from sklearn.cluster.bicluster import SpectralBiclustering
from sklearn.cluster.bicluster import SpectralCoclustering
from sklearn.metrics import consensus_score

import scipy.linalg
from functools import partial
%pylab
%matplotlib inline
import matplotlib as mpl
import matplotlib.gridspec as gridspec
import scipy.sparse as sparse
from scipy import stats
from __future__ import print_function
import gc
from tqdm import tqdm_notebook as tqdm
from __future__ import division
import pandas as pd
import seaborn as sb
import os
from scipy.spatial import distance
from scipy.cluster import hierarchy
from pandas import DataFrame

import os, sys, datetime
LF_CODE_PATH = os.path.expanduser('~/projects/LFAnalyze/code')
FT_CODE_PATH = os.path.expanduser('~/projects/fishTrax/code/analysis/')
FD_CODE_PATH = os.path.expanduser('~/projects/fish_despair_notebooks/src/')
sys.path.insert(0,LF_CODE_PATH)
sys.path.insert(0,FT_CODE_PATH)
sys.path.insert(0,FD_CODE_PATH)

import passivity_2p_imaging_utils as p2putils
import visualization_utils as vizutil
reload(p2putils)
tmp_dir = '/tmp/'
all_data = p2putils.get_all_datasets(tmp_dir=tmp_dir)

f = all_data['e'][2]

# import tensorflow as tf
# tf.enable_eager_execution()


# df, sig = f.get_cnmf_roi_table_and_signals()
M = f.get_signals_raw(z=None)

# data = np.load("../cnmf_f01555.npz")
# cnmf = data['cnmf'].astype(np.float32)
# raw = data['raw'].astype(np.float32)
# del data

def moving_average(a, n=3):
    # a is a signal
    ret = np.cumsum(a,0) # sum over time
    ret[n:] = ret[n:] - ret[:-n] # diff of n samples back
    rm = ret[n - 1:] / n
    pad_start = np.full((n-1,rm.shape[1]), rm[0])
    return np.vstack([pad_start, rm])

def ewma(data,span):
    "exponential weighted moving average."
    df = DataFrame(data)
    return df.ewm(span).mean().values

def df_f(x,ma_window=6,span=6):
    u = moving_average(x,ma_window)
    return ewma((x - u)/u, span)

# neurons = sig.T
neurons = M.T

neuron_ids = np.sort(np.argsort(neurons.std(0))[-5000:])
df = f.get_roi_table()
df = df.iloc[neuron_ids]
df.reset_index(drop=True, inplace=True)
neurons = neurons[:,neuron_ids]

neurons = T.from_numpy(df_f(neurons).astype(np.float32)).cuda()
neurons = (neurons - neurons.mean(0))/(neurons.std(0)+1e-8)

/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


Using matplotlib backend: agg
Populating the interactive namespace from numpy and matplotlib


/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


NameError: name 'T' is not defined

In [ ]:
df.head()

In [5]:
tf.ones([5,2])

<tf.Tensor 'ones_2:0' shape=(5, 2) dtype=float32>

In [3]:
tf.ones([6, 1]).shape

TensorShape([Dimension(6), Dimension(1)])

In [43]:
tf.Tensor(np.arange(5))

TypeError: __init__() takes exactly 4 arguments (2 given)

In [56]:
def construct_pairwise_batches(X, lag=1, bias=True):
    "constructs batch x nT-lag x nBeta. i*ncol+j index is granger model of influence from i->j."
    nT, N = X.shape
    granger = np.ones([N**2,nT-lag,2*lag+bias])
    null = np.ones([N**2,nT-lag,lag+bias])
    newY = np.ones([N**2,nT-lag,1])
    Y = X[lag:]
    # time x neuron x lag
    l = 0
    X_lag = np.concatenate([X[l:-(lag-l)][:,:,None] for l in reversed(range(lag))], 2)
    biasTerm = np.ones([nT-lag,1])
    for n1 in range(N):
        for n2 in range(N):
            if bias:
                grow = np.concatenate([X_lag[:,n1], X_lag[:,n2], biasTerm], 1)
                nrow = np.concatenate([X_lag[:,n2], biasTerm], 1)
            else:
                grow = np.concatenate([X_lag[:,n1], X_lag[:,n2]], 1)
                nrow = np.concatenate([X_lag[:,n2]], 1)
            granger[n1*N+n2] = grow
            null[n1*N+n2] = nrow
            newY[n1*N+n2] = Y[:,[n2]]
    return newY, granger, null

x = np.random.rand(7,3)
Y, x_granger, x_null = construct_pairwise_batches(x,1, True)

def tf_ols(X, Y):
    with tf.Session() as sess:
        gbeta = tf.matrix_solve_ls(tf.convert_to_tensor(X), tf.convert_to_tensor(Y), fast=False)
        g_residuals = tf.reduce_sum(((tf.matmul(X,gbeta) - Y)**2)[:,:,0], 1)
        gbeta, g_residuals = sess.run([gbeta, g_residuals])
    return gbeta, g_residuals

gbeta, g_residuals = tf_ols(x_granger, Y)

# g_residuals_single = ((T.matmul(x_granger[0],gbeta_single) - Y[0])**2).sum()
# n_residuals = T.matmul(x_null,nbeta)[:,:,0].sum(1)

np_beta, _ = np.linalg.lstsq(x_granger[0], Y[0], rcond=None)[0:2]
np_res = sum((np.matmul(x_granger[0],np_beta) - Y[0])**2)
print("tf vs numpy")
print(gbeta[0,:,0],"\n", np_beta[:,0])
print("residuals", float(g_residuals[0]), np_res)

tf vs numpy
[-0.17200707 -0.17200707  0.44842334] 
 [-0.17200707 -0.17200707  0.44842334]
residuals 0.473876291477 0.47387629147726723


In [64]:
x = np.random.rand(2800,1000)
Y, x_granger, x_null = construct_pairwise_batches(x,1, True)

KeyboardInterrupt: 

In [ ]:
%time
gbeta, g_residuals = tf_ols(x_granger, Y)

In [63]:
gbeta.shape

(10000, 3, 1)

In [35]:
np.concatenate([1,2,3])

ValueError: zero-dimensional arrays cannot be concatenated

In [5]:
def ols_tf(X, Y):
    
x = T.rand(7,3)
Y, x_granger, x_null = construct_pairwise_batches(x,1, True)

gbeta = ols(x_granger, Y)
gbeta_single = ols_single(x_granger[0], Y[0])
nbeta = ols(x_null, Y)
g_residuals = ((T.matmul(x_granger,gbeta) - Y)**2)[:,:,0].sum(1)
g_residuals_single = ((T.matmul(x_granger[0],gbeta_single) - Y[0])**2).sum()
n_residuals = T.matmul(x_null,nbeta)[:,:,0].sum(1)

np_beta, _ = np.linalg.lstsq(x_granger[0].numpy(), Y[0].numpy(), rcond=None)[0:2]
np_res = sum((np.matmul(x_granger[0].numpy(),np_beta) - Y[0].numpy())**2)
print("pytorch vs numpy")
print("batch", gbeta[0,:,0].numpy(),"\n", np_beta[:,0])
print("single", gbeta_single[:,0].numpy(),"\n", np_beta[:,0])
print("batch residuals", float(g_residuals[0]), np_res)
print("single residuals", g_residuals_single, np_res)

pytorch vs numpy
batch [1.9759694 1.9759694 3.7562916] 
 [-0.1206158  -0.1206158   0.76239705]
single [-0.1206159  -0.1206159   0.76239717] 
 [-0.1206158  -0.1206158   0.76239705]
batch residuals 184.001693726 0.6442907
single residuals tensor(0.6443) 0.6442907


In [8]:
np_beta, _ = np.linalg.lstsq(x_granger[0].numpy(), Y[0].numpy(), rcond=None)[0:2]

TypeError: must be real number, not NoneType

In [10]:
x_granger[0].numpy()

array([[ 0.15112823,  0.15112823,  1.        ],
       [ 0.76432306,  0.76432306,  1.        ],
       [ 0.28615254,  0.28615254,  1.        ],
       [ 0.37601125,  0.37601125,  1.        ],
       [ 0.20394307,  0.20394307,  1.        ],
       [ 0.20240605,  0.20240605,  1.        ]], dtype=float32)

In [9]:
Y[0].numpy()

array([[ 0.76432306],
       [ 0.28615254],
       [ 0.37601125],
       [ 0.20394307],
       [ 0.20240605],
       [ 0.83572507]], dtype=float32)

In [6]:
x_granger

tensor([[[0.5681, 0.5681, 1.0000],
         [0.7721, 0.7721, 1.0000],
         [0.6990, 0.6990, 1.0000],
         [0.5298, 0.5298, 1.0000],
         [0.9802, 0.9802, 1.0000],
         [0.0606, 0.0606, 1.0000]],

        [[0.5681, 0.9459, 1.0000],
         [0.7721, 0.0199, 1.0000],
         [0.6990, 0.5527, 1.0000],
         [0.5298, 0.8194, 1.0000],
         [0.9802, 0.7203, 1.0000],
         [0.0606, 0.0549, 1.0000]],

        [[0.5681, 0.2576, 1.0000],
         [0.7721, 0.4094, 1.0000],
         [0.6990, 0.8284, 1.0000],
         [0.5298, 0.4898, 1.0000],
         [0.9802, 0.9050, 1.0000],
         [0.0606, 0.9853, 1.0000]],

        [[0.9459, 0.5681, 1.0000],
         [0.0199, 0.7721, 1.0000],
         [0.5527, 0.6990, 1.0000],
         [0.8194, 0.5298, 1.0000],
         [0.7203, 0.9802, 1.0000],
         [0.0549, 0.0606, 1.0000]],

        [[0.9459, 0.9459, 1.0000],
         [0.0199, 0.0199, 1.0000],
         [0.5527, 0.5527, 1.0000],
         [0.8194, 0.8194, 1.0000],
         [0.

In [120]:
x = T.rand(2800,500)
Y, x_granger, x_null = construct_pairwise_batches(x,1, True)

In [122]:
%%time
np_beta, _ = np.linalg.lstsq(x_granger[0], Y[0], rcond=None)[0:2]
np_res = sum((np.matmul(x_granger[0].numpy(),np_beta) - Y[0].numpy())**2)

CPU times: user 5.27 ms, sys: 0 ns, total: 5.27 ms
Wall time: 2.28 ms


In [124]:
np_res

222.04225

In [118]:
3/10000*15000**2/60/60

18.75

In [84]:
np_beta.dtype

dtype('float32')

In [59]:
np.matmul(x_granger[0].numpy(), np_beta).shape


(6, 1)

In [26]:
m.numpy()[0]

array([[0.2073974 , 0.8812977 , 0.7992798 ],
       [0.7542076 , 0.15377986, 0.43977767],
       [0.8672233 , 0.9895245 , 0.3223688 ]], dtype=float32)

In [31]:
m = T.rand(1,3,3)
minv = batch_inverse(m)
ninv = np.linalg.inv(m.numpy()[0])
print(minv[0], ninv)

tensor([[ 2.4396,  4.0179, -4.3811],
        [ 0.5671, 13.9223, -8.8159],
        [-1.0968, -8.1438,  7.1031]]) [[ 2.4396143  4.017931  -4.381108 ]
 [ 0.5670625 13.922265  -8.815909 ]
 [-1.096764  -8.143784   7.1031103]]


In [4]:
def lm(X, Y, bias=True):
    "Fast linear regression"
    Y = Y[:,None]
    if bias:
        X = T.cat((X,T.ones(X.shape[0],1)),1)
    res = T.mm(T.inverse(T.mm(X.t(),X)), X.t())
    beta = T.mm(res, Y)
    residuals = ((Y-T.mm(X,beta))**2).sum()
    return beta, residuals

x = T.rand(20,5)
y = T.rand(20)
A = np.hstack([x.numpy(), np.ones([20,1])])
beta, residuals = lm(x, y)
np_beta, np_res = np.linalg.lstsq(A, y, rcond=None)[0:2]
print(beta[:,0].numpy(),"\n", np_beta)
print(float(residuals), np_res)

[ 0.08088723 -0.01808028 -0.33222944 -0.5440871  -0.03814803  0.8367273 ] 
 [ 0.08088763 -0.01807992 -0.33222925 -0.54408743 -0.03814821  0.83672717]
1.2584692239761353 [1.25846926]


In [ ]:
# multiple models
def lm_mult(X, Y, bias=True):
    "Fast linear regression"
    Y = Y[:,None]
    if bias:
        X = T.cat((X,T.ones(X.shape[0],1)),1)
    res = T.matmul(T.inverse(T.matmul(X.t(),X)), X.t())
    beta = T.matmul(res, Y)
    residuals = ((Y-T.matmul(X,beta))**2).sum()
    return beta, residuals

x = T.rand(20,5)
y = T.rand(20)
A = np.hstack([x.numpy(), np.ones([20,1])])
# beta, residuals = lm(x, y)
np_beta, np_res = np.linalg.lstsq(A, y, rcond=None)[0:2]
print(beta[:,0].numpy(),"\n", np_beta)
print(float(residuals), np_res)

In [ ]:
T.version.cuda

In [ ]:
np_beta.shape

In [ ]:
x = T.rand(2,10,10)
y = T.rand(2,10,1)
beta, _ = T.gesv(y,x)
residuals = T.matmul(x,beta)[:,:,0].sum(1)
print(residuals.shape)
np_beta, np_res = np.linalg.lstsq(x[0], y[0], rcond=None)[0:2]
print(beta[0,:,0].numpy(),"\n", np_beta[:,0])
print(float(residuals[0]), np_res)

In [ ]:
np_res

In [ ]:
T.inverse(x)

In [ ]:
x = T.rand(2800,2)
y = T.rand(2800)
A = np.hstack([x.numpy(), np.ones([len(x),1])])

In [ ]:
5000**2/1000/60

In [ ]:
%%timeit
np_beta, np_res = np.linalg.lstsq(A, y, rcond=None)[0:2]

In [ ]:
y.shape

In [ ]:
(T.mm(T.from_numpy(A.astype(np.float32)),beta)).shape

In [ ]:
beta

In [ ]:
np_beta

In [ ]:
residuals

In [ ]:
np_res

In [ ]:
plt.matshow(A)

In [ ]:
A.shape

In [ ]:
x = T.rand(10,10)
plt.matshow(torch.mm(torch.inverse(x), x))

In [ ]:
G = granger(X,Y)

In [ ]:
plt.hist(G.cpu().numpy()[:])

In [ ]:
plt.imshow(G)

In [121]:
import gc

gc.collect()

T.cuda.empty_cache()

In [ ]:
def cm_cycle(i):
    i = i - 1
    cmap = plt.get_cmap("tab10")
    color = cmap(i%10)
    if i%10==7:
        color = (color[0],color[1],color[2]+0.4)
    return color

In [ ]:
row_corr = corrcoef(G).cpu().numpy()

row_linkage = hierarchy.linkage(
    distance.pdist(row_corr), method='ward')

In [ ]:
col_corr = corrcoef(G.t().detach()).cpu().numpy()
col_linkage = hierarchy.linkage(
    distance.pdist(col_corr.T), method='ward')

In [ ]:
g_nclust = 10
row_clusters = hierarchy.fcluster(row_linkage,g_nclust,criterion='maxclust')
col_clusters = hierarchy.fcluster(col_linkage,g_nclust,criterion='maxclust')

In [ ]:
vmax = max(np.abs(np.percentile(G,0.01)), np.abs(np.percentile(G,0.99)))

In [ ]:
cm = sb.clustermap(G.cpu().numpy(), row_linkage=row_linkage, row_colors=[cm_cycle(c) for c in row_clusters],
        col_linkage=col_linkage, col_colors=[cm_cycle(c) for c in col_clusters],
        figsize=(20, 20),cmap="RdBu_r", vmin=-vmax,vmax=vmax)

In [ ]:
Z = [0,2,4,6,8,10]
# Z = [4]
nZ = len(Z)
back_img = []
for z in Z:
    back_img.append(np.power(f.get_tif_rasl_as_vol(z,range(1,200)).mean(axis=2),.4))
    

In [ ]:
# nrows = len(row_clust_to_plot)
# nZ = 1
nrows = g_nclust
plt.subplots(nrows,nZ, figsize=[8*nZ,4*nrows])
for ic, clust in enumerate(range(1, g_nclust+1)):
# for ic,clust in enumerate(row_clust_to_plot):
    for iz, z in enumerate(Z):
        #Select rois in raphe in this slices, and get their coordinates.
#         coords = df[(row_clusters==clust)].coords
        coords = df[(row_clusters==clust) & (df.z==z)].coords
#         poly_coords = df[(row_clusters==clust) & (df.z==z)].poly
#         poly_coords = df[(row_clusters==clust)].poly
#         coords = [np.round(poly[~np.isnan(poly).any(axis=1)])[:,(1,0)].astype(int) for poly in poly_coords]
        plt.subplot(nrows,nZ,ic*nZ+iz+1)
        #Overlay the ROIs on the background image and display:
        # hack iz hardcode
        img = vizutil.overlay_coords(back_img[iz], coords, list(cm_cycle(clust)[:3]), alpha=1)
        plt.imshow(img,interpolation='nearest')
        plt.title("Row cluster {}, z={}".format(clust,z),fontsize=18)

In [ ]:
# nrows = len(col_clust_to_plot)
plt.subplots(nrows,nZ, figsize=[8*nZ,4*nrows])
for ic,clust in enumerate(range(1,g_nclust+1)):
# for ic,clust in enumerate(col_clust_to_plot):
    for iz, z in enumerate(Z):
        #Select rois in raphe in this slices, and get their coordinates.
        coords = df[(col_clusters==clust) & (df.z==z)].coords
#         poly_coords = df[(row_clusters==clust) & (df.z==z)].poly
#         coords = df[(col_clusters==clust)].coords
#         poly_coords = df[(col_clusters==clust)].poly
#         coords = [np.round(poly[~np.isnan(poly).any(axis=1)])[:,(1,0)].astype(int) for poly in poly_coords]
        plt.subplot(nrows,nZ,ic*nZ+iz+1)
        #Overlay the ROIs on the background image and display:
        img = vizutil.overlay_coords(back_img[iz], coords, list(cm_cycle(clust)[:3]), alpha=1)
        plt.imshow(img,interpolation='nearest')
        plt.title("Col cluster {}, z={}".format(clust,z),fontsize=18)

In [ ]:
s = 0
e = 440
fig, ax = plt.subplots(1,2, figsize=(5,100))
ax[0].imshow(X[slice(*idx_permute(s,e,exclude_idx))].T)
ax[1].imshow(x_fish[s:e].T)

In [ ]:
colidx = col_clusters==34
rowidx = row_clusters==7
plt.matshow(G[rowidx][:,colidx],vmin=-0.1,vmax=0.1)
plt.colorbar()

In [ ]:
ind = slice(*idx_permute(2400,2800, exclude_idx))
# ind = slice(*idx_permute(0,400, exclude_idx))

In [ ]:
# model = Granger(len(neuron_ids),1,bias=False)
model = Granger(X.shape[1],1,bias=False)
G = model.G.data[0].detach().cpu().numpy()
# trunc_fish = x_fish[0:400]
# trunc_fish = x_fish[440:840]
# trunc_fish = x_fish[840:1240]
# trunc_fish = x_fish[1240:1640]
# trunc_fish = x_fish[1640:2040]
# trunc_fish = x_fish[1640:2440]
# trunc_fish = x_fish[640:840]
# trunc_fish = x_fish[2400:2800]
# trunc_fish = x_fish[2600:2800]
# trunc_fish = x_fish
# data = T.utils.data.TensorDataset(T.from_numpy(X[ind,None,neuron_ids]).cuda(),
#                                   T.from_numpy(Y[ind,neuron_ids]).cuda())
data = T.utils.data.TensorDataset(T.from_numpy(X[ind,None]).cuda(),
                                  T.from_numpy(Y[ind]).cuda())
l1 = 5e-6
# train(model,data,10,l1=l1,batch_size=4) # max(bz)==4 for all neurons
train(model,data,3,l1p=1e-0,lr=1e-1,batch_size=1,verbose=False)
train(model,data,15,l1p=5e-2,lr=1e-3,batch_size=1,verbose=False)
G = model.G.data[0].detach().cpu().numpy()